## Besoin client 3

In [2]:
import itertools
import math

import pandas as pd
import sklearn as sk
import numpy as np

In [4]:
DATA_FILE = '../data/export_IA.csv'
DATA_FILE = '/home/skoh/Downloads/share/export_IA.csv'

df = pd.read_csv(DATA_FILE, parse_dates=[1])
# df = df.iloc[: , 1:] # remove the id column
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
0,3791472,2024-06-24 01:41:57,26.13177,-92.04046,0.0,18.7,1.000000,LUCIUS SPAR,IMO1108561,WQXP40,80,0,0.0,0.0,0.000000,80.000000,A
1,3791472,2024-06-24 02:00:28,26.13173,-92.04054,0.0,11.9,1.000000,LUCIUS SPAR,IMO1108561,WQXP40,80,0,0.0,0.0,0.000000,80.000000,A
2,3791472,2024-06-24 02:29:18,26.13160,-92.04039,0.1,125.6,1.000000,LUCIUS SPAR,IMO1108561,WQXP40,80,0,0.0,0.0,0.000000,80.000000,A
3,3791472,2024-06-24 02:34:48,26.13176,-92.04038,0.0,314.4,1.000000,LUCIUS SPAR,IMO1108561,WQXP40,80,0,0.0,0.0,0.000000,80.000000,A
4,3791472,2024-06-24 16:54:58,26.13173,-92.04034,0.1,167.6,1.000000,LUCIUS SPAR,IMO1108561,WQXP40,80,0,0.0,0.0,0.000000,80.000000,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5302028,982158086,2024-06-29 13:32:53,26.08813,-80.11431,0.9,37.6,129.607425,BEYOND TB06,IMO0000000,9HA5296,60,0,18.0,6.0,1.554094,59.305000,B
5302029,982158086,2024-06-29 13:35:53,26.08825,-80.11431,1.6,160.9,143.978790,BEYOND TB06,IMO0000000,9HA5296,60,0,18.0,6.0,1.539473,58.745000,B
5302030,982158086,2024-06-29 13:41:53,26.08817,-80.11435,0.1,340.9,151.515081,BEYOND TB06,IMO0000000,9HA5296,60,0,18.0,6.0,1.621767,58.992778,B
5302031,983110342,2024-06-28 15:26:09,24.55833,-81.80930,0.2,360.0,197.482518,TB 8 VALIANT LADY,NaN,C6ER7,60,0,14.0,4.0,1.950372,63.025000,B


On regarde si il y a des `NA`

In [5]:
df.isna().any()

MMSI                False
BaseDateTime        False
LAT                 False
LON                 False
SOG                 False
COG                 False
Heading             False
VesselName          False
IMO                  True
CallSign             True
VesselType          False
Status              False
Length              False
Width               False
Draft               False
Cargo               False
TransceiverClass    False
dtype: bool

C'est le cas, mais il s'agit des colonnes `IMO` et `CallSign`, qui servent juste à l'identification des bateaux  
Ce n'est donc pas nécessaire de supprimer la ligne, on peut juste l'ignorer  
Ajoutons une colonne `next_LAT` et `next_LON`

In [6]:
df = df.sort_values(by=['MMSI', 'BaseDateTime'])
df.reset_index(drop=True, inplace=True)

df["NextLAT"] = None
df["NextLON"] = None
df["NextTime"] = None

# def fn(j, rows):
#     print(rows)
#     for i in rows.rolling(window=2):
#         print(i)

# all_boats = map(fn, df.groupby(by="MMSI"))

# for i in df.groupby(by="MMSI"):
    # print(len(i))

# Super slow and super bad, but idk how to do better
total = len(pd.unique(df["MMSI"]))
for i in range(len(df)-1):

    # If new boat (df is sorted)
    if(df.at[i, "MMSI"] != df.at[i+1, "MMSI"]):
        # print(f"Skipped {total}")
        total -= 1
        continue

    if (df.at[i, "SOG"] < 0.3):
        continue

    df.at[i, "NextLAT"] = df.at[i+1, "LAT"]
    df.at[i, "NextLON"] = df.at[i+1, "LON"]
    df.at[i, "NextTime"] = df.at[i+1, "BaseDateTime"] - df.at[i, "BaseDateTime"]

# Remove columns where we can't find a next location
df = df[df["NextTime"].notna()]

df.shape

(2360582, 20)

### First tentative
Approche simple, sans optimisation, juste application d'un modèle

In [7]:
df["NextTime"] = df["NextTime"].apply(pd.Timedelta.total_seconds)
df["VesselType"] = df["VesselType"].astype(str)

In [8]:
# https://www.geeksforgeeks.org/dsa/haversine-formula-to-find-distance-between-two-points-on-a-sphere/
def haversine(lat1, lon1, lat2, lon2):
    
    # distance between latitudes
    # and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0

    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c * 1000

In [9]:
x_ = pd.get_dummies(df[["LAT", "LON", "SOG", "COG", "Heading", "VesselType", "NextTime"]])
y_ = df[["NextLAT", "NextLON"]]

def evaluate_model(x_, y_, md):
    x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(x_, y_)
    
    reg = md.fit(x_train, y_train)
    pred = reg.predict(x_test)

    print("a")
    
    return sk.metrics.mean_absolute_error(y_test, pred)

## Results
---

In [11]:
lnr = sk.linear_model.LinearRegression()

dst = np.mean([evaluate_model(x_, y_, lnr) for _ in range(20)])
f'Incertitude: {haversine(dst, dst, 0, 0):.2f}m'

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


'Incertitude: 619.19m'

In [ ]:
sgd = sk.pipeline.make_pipeline(
    sk.preprocessing.StandardScaler(),
    # sk.linear_model.LinearRegression()
    # sk.multioutput.MultiOutputRegressor(
        # sk.svm.LinearSVR(verbose=1, max_iter=10_000)
        # sk.ensemble.HistGradientBoostingRegressor()
    # )
    # sk.multioutput.MultiOutputRegressor(sk.linear_model.SGDRegressor())
    # sk.preprocessing.PolynomialFeatures(degree=4), # looks bad and slow
    sk.ensemble.RandomForestRegressor() # GOOOOOOOOD
    # sk.linear_model.LinearRegression()
)

x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(x_, y_)



# reg = md.fit(x_train, y_train)
# pred = reg.predict(x_test)
# print('a')

# return sk.metrics.mean_absolute_error(y_test, pred)

dst = np.mean([evaluate_model(x_, y_, sgd) for _ in range(1)])
f'Incertitude: {haversine(dst, dst, 0, 0):.2f}m'

---

LinearRegression good  
Ridge good  
SVR ? (giga slow and only decent)  
MinMaxScaler + ?????  
RandomForest is goated

## Recherches

Il faut prédire la position future d'un bateau (à 5min, 10min, 15min...)

Quel type de modèle ?  
Quels variables significatives ?  
Strats pour opti ?  


Observations de la partie précédente:
- Bateaux passagers proche des cotes, bateaux cargo/tanker loin des cotes
- Forte correlation

Bateau  

---


Je prends un point de start  
Model1: détecte points alentours  
Model2: détecte quel point fait sens (same type de bateau, direction...)  
Model3: quels sont les points suivants des sames types de bateaux  